
#  Training with customdataset 1

In [7]:
# Imports
import pathlib
from transformations import Compose, AlbuSeg2d, DenseTarget 
from transformations import MoveAxis, Normalize01, Resize
from sklearn.model_selection import train_test_split
from customdatasets import SegmentationDataSet
import torch
from unet import UNet
from trainer import Trainer
from torch.utils.data import DataLoader
import albumentations
from catalyst.contrib.nn import DiceLoss, RAdam, Lookahead, OneCycleLRWithWarmup

# root directory
root = pathlib.Path.cwd() / 'Carvana'
def get_filenames_of_path(path: pathlib.Path, ext: str = '*'):
    """Returns a list of files in a directory/path. Uses pathlib."""
    filenames = [file for file in path.glob(ext) if file.is_file()]
    return filenames

# input and target files
inputs = get_filenames_of_path(root / 'Input')
targets = get_filenames_of_path(root / 'Target')

# training transformations and augmentations
transforms_training = Compose([
    Resize(input_size=(128, 128, 3), target_size=(128, 128)),
    AlbuSeg2d(albu=albumentations.HorizontalFlip(p=0.5)),
    DenseTarget(),
    MoveAxis(),
    Normalize01()
])

# validation transformations
transforms_validation = Compose([
    Resize(input_size=(128, 128, 3), target_size=(128, 128)),
    DenseTarget(),
    MoveAxis(),
    Normalize01()
])


# random seed
random_seed = 42

# split dataset into training set and validation set
train_size = 0.8  # 80:20 split

inputs_train, inputs_valid = train_test_split(
    inputs,
    random_state=random_seed,
    train_size=train_size,
    shuffle=True)

targets_train, targets_valid = train_test_split(
    targets,
    random_state=random_seed,
    train_size=train_size,
    shuffle=True)

# inputs_train, inputs_valid = inputs[:80], inputs[80:]
# targets_train, targets_valid = targets[:80], targets[:80]

# dataset training
dataset_train = SegmentationDataSet(inputs=inputs_train,
                                    targets=targets_train,
                                    transform=transforms_training)

# dataset validation
dataset_valid = SegmentationDataSet(inputs=inputs_valid,
                                    targets=targets_valid,
                                    transform=transforms_validation)

# dataloader training
dataloader_training = DataLoader(dataset=dataset_train,
                                 batch_size=2,
                                 shuffle=True)

# dataloader validation
dataloader_validation = DataLoader(dataset=dataset_valid,
                                   batch_size=2,
                                   shuffle=True)

In [8]:
%gui qt
# from visual import Input_Target_Pair_Generator
# from visual import show_input_target_pair_napari
#
# gen_t = Input_Target_Pair_Generator(dataloader_training, rgb=True)
# gen_v = Input_Target_Pair_Generator(dataloader_validation, rgb=True)
# show_input_target_pair_napari(gen_t, gen_v)

In [7]:
# device
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# model
model = UNet(in_channels=3,
             out_channels=2,
             n_blocks=4,
             start_filters=32,
             activation='relu',
             normalization='batch',
             conv_mode='same',
             dim=2).to(device)

# criterion
criterion = torch.nn.NLLLoss()

# optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# trainer
trainer = Trainer(model=model,
                  device=device,
                  criterion=criterion,
                  optimizer=optimizer,
                  training_DataLoader=dataloader_training,
                  validation_DataLoader=dataloader_validation,
                  lr_scheduler=None,
                  epochs=4,
                  epoch=0,
                  notebook=True)

# start training
training_losses, validation_losses, lr_rates = trainer.run_trainer()

Progress:   0%|          | 0/4 [00:00<?, ?it/s]

Training:   0%|          | 0/183 [00:00<?, ?it/s]

RuntimeError: only batches of spatial targets supported (3D tensors) but got targets of dimension: 4

# Training with customdataset 2

In [1]:
# Imports
import pathlib
from transformations import Compose, AlbuSeg2d, DenseTarget 
from transformations import MoveAxis, Normalize01, Resize
from sklearn.model_selection import train_test_split
from customdatasets2 import SegmentationDataSet
import torch
from unet import UNet
from trainer import Trainer
from torch.utils.data import DataLoader
import albumentations

# root directory
root = pathlib.Path.cwd() / 'Carvana'
def get_filenames_of_path(path: pathlib.Path, ext: str = '*'):
    """Returns a list of files in a directory/path. Uses pathlib."""
    filenames = [file for file in path.glob(ext) if file.is_file()]
    return filenames

# input and target files
inputs = get_filenames_of_path(root / 'Input')
targets = get_filenames_of_path(root / 'Target')

# pre-transformations
pre_transforms = Compose([
    Resize(input_size=(128, 128, 3), target_size=(128, 128)),
])
# training transformations and augmentations
transforms_training = Compose([
    AlbuSeg2d(albu=albumentations.HorizontalFlip(p=0.5)),
    DenseTarget(),
    MoveAxis(),
    Normalize01()
])
# validation transformations
transforms_validation = Compose([
    Resize(input_size=(128, 128, 3), target_size=(128, 128)),
    DenseTarget(),
    MoveAxis(),
    Normalize01()
])
# random seed
random_seed = 42

# split dataset into training set and validation set
train_size = 0.8  # 80:20 split

inputs_train, inputs_valid = train_test_split(
    inputs,
    random_state=random_seed,
    train_size=train_size,
    shuffle=True)

targets_train, targets_valid = train_test_split(
    targets,
    random_state=random_seed,
    train_size=train_size,
    shuffle=True)

# inputs_train, inputs_valid = inputs[:80], inputs[80:]
# targets_train, targets_valid = targets[:80], targets[:80]

# dataset training
dataset_train = SegmentationDataSet(inputs=inputs_train,
                                    targets=targets_train,
                                    transform=transforms_training,
                                    use_cache=True,
                                    pre_transform=pre_transforms)

# dataset validation
dataset_valid = SegmentationDataSet(inputs=inputs_valid,
                                    targets=targets_valid,
                                    transform=transforms_validation,
                                    use_cache=True,
                                    pre_transform=pre_transforms)

# dataloader training
dataloader_training = DataLoader(dataset=dataset_train,
                                 batch_size=2,
                                 shuffle=True)

# dataloader validation
dataloader_validation = DataLoader(dataset=dataset_valid,
                                   batch_size=2,
                                   shuffle=True)

x, y = next(iter(dataloader_training))

print(f'x = shape: {x.shape}; type: {x.dtype}')
print(f'x = min: {x.min()}; max: {x.max()}')
print(f'y = shape: {y.shape}; class: {y.unique()}; type: {y.dtype}')

Caching:   0%|          | 0/366 [00:00<?, ?it/s]

Caching:   0%|          | 0/92 [00:00<?, ?it/s]

x = shape: torch.Size([2, 3, 128, 128]); type: torch.float32
x = min: 0.0; max: 1.0
y = shape: torch.Size([2, 128, 128, 3]); class: tensor([0, 1, 2, 3, 4, 5]); type: torch.int64


In [4]:
%gui qt
from visual import Input_Target_Pair_Generator
from visual import show_input_target_pair_napari

gen_t = Input_Target_Pair_Generator(dataloader_training, rgb=True)
gen_v = Input_Target_Pair_Generator(dataloader_validation, rgb=True)
show_input_target_pair_napari(gen_t, gen_v)

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 63.50000000000001, 63.50000000000001), zoom=3.822440944881889, angles=(0.0, 0.0, 90.0), interactive=True), cursor=Cursor(position=(0.0, 0.0, 0.0), scaled=True, size=10, style='standard'), dims=Dims(ndim=3, ndisplay=2, last_used=2, range=((0.0, 127.0, 1.0), (0.0, 127.0, 1.0), (0.0, 127.0, 1.0)), current_step=(0, 0, 0), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(enabled=False, stride=1, shape=(-1, -1)), layers=[<Image layer 'input_training' at 0x1b732eb36d0>, <Labels layer 'target_training' at 0x1b7330f78b0>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position='bottom_right'), active_layer=<Labels layer 'target_training' at 0x1b7330f78b0>, help='enter paint or fill mode to edit labels', status='target_training [0 0 0]: 0', theme='dark', title='napari', mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_wheel_callbacks=[<function dim

In [12]:
# device
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# model
model = UNet(in_channels=3,
             out_channels=2,
             n_blocks=4,
             start_filters=32,
             activation='relu',
             normalization='batch',
             conv_mode='same',
             dim=2).to(device)

# criterion
# criterion = torch.nn.CrossEntropyLoss()
criterion = DiceLoss()


# optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# trainer
trainer = Trainer(model=model,
                  device=device,
                  criterion=criterion,
                  optimizer=optimizer,
                  training_DataLoader=dataloader_training,
                  validation_DataLoader=dataloader_validation,
                  lr_scheduler=None,
                  epochs=10,
                  epoch=0,
                  notebook=True)

# start training
training_losses, validation_losses, lr_rates = trainer.run_trainer()

Progress:   0%|          | 0/10 [00:00<?, ?it/s]

Training:   0%|          | 0/183 [00:00<?, ?it/s]

AssertionError: targets(shape torch.Size([2, 128, 128, 3])) and outputs(shape torch.Size([2, 2, 128, 128])) must have the same shape

# Training with customdataset 3

In [1]:
# Imports
import pathlib
from transformations import Compose, AlbuSeg2d, DenseTarget 
from transformations import MoveAxis, Normalize01, Resize
from sklearn.model_selection import train_test_split
from customdatasets3 import SegmentationDataSet
import torch
from unet import UNet
from trainer import Trainer
from torch.utils.data import DataLoader
import albumentations

# root directory
root = pathlib.Path.cwd() / 'Carvana'
def get_filenames_of_path(path: pathlib.Path, ext: str = '*'):
    """Returns a list of files in a directory/path. Uses pathlib."""
    filenames = [file for file in path.glob(ext) if file.is_file()]
    return filenames

# input and target files
inputs = get_filenames_of_path(root / 'Input')
targets = get_filenames_of_path(root / 'Target')

# pre-transformations
pre_transforms = Compose([
    Resize(input_size=(128, 128, 3), target_size=(128, 128)),
])
# training transformations and augmentations
transforms_training = Compose([
    Resize(input_size=(128, 128, 3), target_size=(128, 128)),
    AlbuSeg2d(albu=albumentations.HorizontalFlip(p=0.5)),
    DenseTarget(),
    MoveAxis(),
    Normalize01()
])
# validation transformations
transforms_validation = Compose([
    DenseTarget(),
    MoveAxis(),
    Normalize01()
])
# random seed
random_seed = 42

# split dataset into training set and validation set
train_size = 0.8  # 80:20 split

inputs_train, inputs_valid = train_test_split(
    inputs,
    random_state=random_seed,
    train_size=train_size,
    shuffle=True)

targets_train, targets_valid = train_test_split(
    targets,
    random_state=random_seed,
    train_size=train_size,
    shuffle=True)

# inputs_train, inputs_valid = inputs[:80], inputs[80:]
# targets_train, targets_valid = targets[:80], targets[:80]

# dataset training
dataset_train = SegmentationDataSet(inputs=inputs_train,
                                    targets=targets_train,
                                    transform=transforms_training,
                                    use_cache=True,
                                    pre_transform=pre_transforms)

# dataset validation
dataset_valid = SegmentationDataSet(inputs=inputs_valid,
                                    targets=targets_valid,
                                    transform=transforms_validation,
                                    use_cache=True,
                                    pre_transform=pre_transforms)

# dataloader training
dataloader_training = DataLoader(dataset=dataset_train,
                                 batch_size=2,
                                 shuffle=True)

# dataloader validation
dataloader_validation = DataLoader(dataset=dataset_valid,
                                   batch_size=2,
                                   shuffle=True)

In [6]:

# device
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device =torch.device('cpu')

# model
model = UNet(in_channels=3,
             out_channels=2,
             n_blocks=4,
             start_filters=32,
             activation='relu',
             normalization='batch',
             conv_mode='same',
             dim=2).to(device)

# criterion
#criterion = torch.nn.CrossEntropyLoss()
criterion = DiceLoss()
# criterion = torch.

# optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# trainer
trainer = Trainer(model=model,
                  device=device,
                  criterion=criterion,
                  optimizer=optimizer,
                  training_DataLoader=dataloader_training,
                  validation_DataLoader=dataloader_validation,
                  lr_scheduler=None,
                  epochs=100,
                  epoch=0,
                  notebook=True)

# start training
training_losses, validation_losses, lr_rates = trainer.run_trainer()

Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Training:   0%|          | 0/183 [00:00<?, ?it/s]

AssertionError: targets(shape torch.Size([2, 128, 128, 3])) and outputs(shape torch.Size([2, 2, 128, 128])) must have the same shape

## Save the model

In [ ]:
# save the model
model_name =  'carvana_model2.pt'
torch.save(model.state_dict(), pathlib.Path.cwd() / model_name)

# Learning rate finder

In [ ]:
# save the model
model_name =  'carvana_model2.pt'
torch.save(model.state_dict(), pathlib.Path.cwd() / model_name)

# Learning rate finder

In [ ]:
# save the model
model_name =  'carvana_model2.pt'
torch.save(model.state_dict(), pathlib.Path.cwd() / model_name)

# Learning rate finder

In [9]:
# device
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# model
model = UNet(in_channels=3,
             out_channels=2,
             n_blocks=4,
             start_filters=32,
             activation='relu',
             normalization='batch',
             conv_mode='same',
             dim=2).to(device)

# criterion
criterion = torch.nn.CrossEntropyLoss()

# optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [ ]:
from lr_rate_finder import LearningRateFinder
lrf = LearningRateFinder(model, criterion, optimizer, device)
lrf.fit(dataloader_training, steps=1000)

In [ ]:
lrf.plot()

# Plot results

In [ ]:
from visual import plot_training
fig = plot_training(training_losses, validation_losses, lr_rates, gaussian=True, sigma=1, figsize=(10, 4))